## Imports, data load, metric function definition

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import BallTree

In [2]:
X_train = np.load('X_train_surge.npz')
Y_train = pd.read_csv('Y_train_surge.csv')
X_test = np.load('X_test_surge.npz')

In [3]:
def split_train_set(X, Y, val_size=0.2, seed=None):
    rng = np.random.default_rng(seed)
    
    nb_examples = len(Y)
    val_examples = int(val_size * nb_examples)
    
    val_indices = rng.choice(nb_examples, size=val_examples, replace=False)
    
    train_indices = np.setdiff1d(
        np.arange(nb_examples),
        val_indices,
        assume_unique=True
    )
    train_indices = rng.permutation(train_indices)
    
    X_train = {}
    X_val = {}
    for feat in X.files:
        X_train[feat] = X[feat][train_indices]
        X_val[feat] = X[feat][val_indices]
    
    return X_train, Y.iloc[train_indices], \
            X_val, Y.iloc[val_indices]

X_train, Y_train, X_val, Y_val = split_train_set(X_train, Y_train, val_size=0.091, seed=1234)

In [4]:
t_slp = X_train['t_slp'] / 3600
t_slp_delta = t_slp - t_slp[:, 0].reshape(-1, 1)
np.allclose(np.round(t_slp_delta), np.round(t_slp_delta)[0])

True

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [6]:
SLP_HEIGHT = SLP_WIDTH = 41
SLP_PER_EX = 40
T_SURGE_NORMALISATION = 240.
SLP_REL_TIMESTAMPS = np.arange(24*5, step=3) / T_SURGE_NORMALISATION


def normalised_tensor(array, mean, std):
        return torch.from_numpy((array - mean) / std)

def preprocessing(X, slp_mean=None, slp_std=None, t_slp_mean=None, t_slp_std=None,
                 surge_mean=None, surge_std=None):
    slp = X['slp'].reshape(-1, SLP_PER_EX, SLP_HEIGHT, SLP_WIDTH)
    slp = np.roll(slp, shift=-11, axis=3)
    if slp_mean is None:
        slp_mean = np.mean(slp)
    if slp_std is None:
        slp_std = np.std(slp)
    slp = normalised_tensor(slp, slp_mean, slp_std)
    
    fst_slp = X['t_slp'][:, 0] / 3600
    fst_slp_tmp = fst_slp.reshape(-1, 1)
    
    def rel_surge_time(index):
        t_surge = X[index] / 3600
        t_surge -= fst_slp_tmp
        return torch.from_numpy(t_surge / T_SURGE_NORMALISATION)

    t_surge1_in = rel_surge_time('t_surge1_input')
    t_surge2_in = rel_surge_time('t_surge2_input')
    t_surge1_out = rel_surge_time('t_surge1_output')
    t_surge2_out = rel_surge_time('t_surge2_output')
    
    if t_slp_mean is None:
        t_slp_mean = np.mean(fst_slp)
    if t_slp_std is None:
        t_slp_std = np.std(fst_slp)
    fst_slp = normalised_tensor(fst_slp, t_slp_mean, t_slp_std)
    
    surge1 = X['surge1_input']
    surge2 = X['surge2_input']
    if surge_mean is None or surge_std is None:
        surges = np.concatenate([surge1, surge2], axis=None)
        surge_mean = np.mean(surges)
        surge_std = np.std(surges)
    surge1_in = normalised_tensor(surge1, surge_mean, surge_std)
    surge2_in = normalised_tensor(surge2, surge_mean, surge_std)
    
    return X['id_sequence'], slp, slp_mean, slp_std, \
            fst_slp, t_slp_mean, t_slp_std, \
            t_surge1_in, t_surge2_in, t_surge1_out, t_surge2_out, \
            surge1_in, surge2_in, surge_mean, surge_std

In [7]:
train_id_seq, train_slp, slp_mean, slp_std, \
train_fst_slp, t_slp_mean, t_slp_std, \
train_t_surge1_in, train_t_surge2_in, train_t_surge1_out, train_t_surge2_out, \
train_surge1_in, train_surge2_in, surge_mean, surge_std = preprocessing(X_train)

In [8]:
val_id_seq, val_slp, _, _, \
val_fst_slp, _, _, \
val_t_surge1_in, val_t_surge2_in, val_t_surge1_out, val_t_surge2_out, \
val_surge1_in, val_surge2_in, _, _ = preprocessing(X_val, slp_mean, slp_std, t_slp_mean, t_slp_std, surge_mean, surge_std)

In [9]:
test_id_seq, test_slp, _, _, \
test_fst_slp, _, _, \
test_t_surge1_in, test_t_surge2_in, test_t_surge1_out, test_t_surge2_out, \
test_surge1_in, test_surge2_in, _, _ = preprocessing(X_test, slp_mean, slp_std, t_slp_mean, t_slp_std, surge_mean, surge_std)

In [10]:
train_Y_id_seq = Y_train['id_sequence'].values
assert np.alltrue(train_Y_id_seq == train_id_seq), 'Data/label index mismatch'

train_surge1_out = Y_train.iloc[:, 1:11].values
train_surge1_out = normalised_tensor(train_surge1_out, surge_mean, surge_std)
train_surge2_out = Y_train.iloc[:, 11:].values
train_surge2_out = normalised_tensor(train_surge2_out, surge_mean, surge_std)

In [11]:
val_Y_id_seq = Y_val['id_sequence'].values
assert np.alltrue(val_Y_id_seq == val_id_seq), 'Data/label index mismatch'

val_surge1_out = Y_val.iloc[:, 1:11].values
val_surge1_out = normalised_tensor(val_surge1_out, surge_mean, surge_std)
val_surge2_out = Y_val.iloc[:, 11:].values
val_surge2_out = normalised_tensor(val_surge2_out, surge_mean, surge_std)

In [329]:
from torch.utils.data import TensorDataset, DataLoader

resnet_versions = {
    18 : models.resnet18,
    34 : models.resnet34,
    50 : models.resnet50,
}

batch_size = 64

train_dataset = TensorDataset(
    train_slp,
#     train_fst_slp,
    train_t_surge1_in,
    train_surge1_in,
    train_t_surge2_in,
    train_surge2_in,
    train_t_surge1_out,
    train_surge1_out,
    train_t_surge2_out,
    train_surge2_out,
)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

val_dataset = TensorDataset(
    val_slp,
#     val_fst_slp,
    val_t_surge1_in,
    val_surge1_in,
    val_t_surge2_in,
    val_surge2_in,
    val_t_surge1_out,
    val_surge1_out,
    val_t_surge2_out,
    val_surge2_out,
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(
    test_slp,
#     test_fst_slp,
    test_t_surge1_in,
    test_surge1_in,
    test_t_surge2_in,
    test_surge2_in,
    test_t_surge1_out,
    test_t_surge2_out
)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class Network(nn.Module):
    
    def __init__(self, resnet_layers, lstm_hidden_size, lstm_layers=1, dropout=.2):
        super().__init__()
        self.resnet = resnet_versions[resnet_layers]()
        self.resnet_layers = resnet_layers
        self.resnet.conv1 = nn.Conv2d(SLP_PER_EX, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#         self.resnet.fc = nn.Linear(self.resnet.fc.in_features, lstm_hidden_size)

        self.lstm_hidden_size = lstm_hidden_size
        self.lstm_layers = lstm_layers
        self.fc0 = nn.Linear(self.resnet.fc.out_features, lstm_hidden_size)
        self.fc1 = nn.Linear(20, 1)
        self.fc2 = nn.Linear(20, 1)
        self.lstm = nn.LSTM(input_size=1, hidden_size=lstm_hidden_size, num_layers=lstm_layers, bias=True, batch_first=True, \
                            dropout=dropout, bidirectional=False, proj_size=1)
        
    def __str__(self):
        return f'resnet{self.resnet_layers}_{self.lstm_hidden_size}_{self.lstm_layers}'
        
    def forward(self, slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out):
        cell = self.resnet(slp)
        cell = self.fc0(cell)
        cell = torch.cat([cell, cell], dim=0).unsqueeze(0)
        c_0 = torch.cat([cell]*self.lstm_layers, dim=0)
        
        surge1 = torch.cat([t_surge1_in, surge1_in], dim=1)
        hidden1 = self.fc1(surge1)
        surge2 = torch.cat([t_surge2_in, surge2_in], dim=1)
        hidden2 = self.fc2(surge2)
        hidden = torch.cat([hidden1, hidden2], dim=0).unsqueeze(0)
        h_0 = torch.cat([hidden]*self.lstm_layers, dim=0)
        
        lstm_input = torch.cat([t_surge1_out, t_surge2_out], dim=0).unsqueeze(2)
        output, _ = self.lstm(lstm_input, (h_0, c_0))
        output = output.squeeze()
        return output[:len(slp)], output[len(slp):]

In [12]:
from torch.utils.data import TensorDataset, DataLoader

resnet_versions = {
    18 : models.resnet18,
    34 : models.resnet34,
    50 : models.resnet50,
}

batch_size = 64

train_dataset = TensorDataset(
    train_slp,
#     train_fst_slp,
    train_t_surge1_in,
    train_surge1_in,
    train_t_surge2_in,
    train_surge2_in,
    train_t_surge1_out,
    train_surge1_out,
    train_t_surge2_out,
    train_surge2_out,
)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

val_dataset = TensorDataset(
    val_slp,
#     val_fst_slp,
    val_t_surge1_in,
    val_surge1_in,
    val_t_surge2_in,
    val_surge2_in,
    val_t_surge1_out,
    val_surge1_out,
    val_t_surge2_out,
    val_surge2_out,
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(
    test_slp,
#     test_fst_slp,
    test_t_surge1_in,
    test_surge1_in,
    test_t_surge2_in,
    test_surge2_in,
    test_t_surge1_out,
    test_t_surge2_out
)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class Network(nn.Module):
    
    def __init__(self, resnet_layers, lstm_hidden_size, lstm_layers=1, dropout=.2):
        super().__init__()
        self.resnet = resnet_versions[resnet_layers]()
        self.resnet_layers = resnet_layers
        self.resnet.conv1 = nn.Conv2d(SLP_PER_EX, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#         self.resnet.fc = nn.Linear(self.resnet.fc.in_features, lstm_hidden_size)

        self.lstm_hidden_size = lstm_hidden_size
        self.lstm_layers = lstm_layers
        self.fc0 = nn.Linear(self.resnet.fc.out_features, lstm_hidden_size)
        self.fc1 = nn.Linear(20, lstm_hidden_size)
        self.fc2 = nn.Linear(20, lstm_hidden_size)
        self.lstm = nn.LSTM(input_size=1, hidden_size=lstm_hidden_size, num_layers=lstm_layers, bias=True, batch_first=True, \
                            dropout=dropout, bidirectional=False, proj_size=1)
        
    def __str__(self):
        return f'resnet{self.resnet_layers}_{self.lstm_hidden_size}_{self.lstm_layers}'
        
    def forward(self, slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out):
        cell = self.resnet(slp)
        cell = self.fc0(cell)
        cell = torch.cat([cell, cell], dim=0).unsqueeze(0)
        
        surge1 = torch.cat([t_surge1_in, surge1_in], dim=1)
        hidden1 = self.fc1(surge1)
        surge2 = torch.cat([t_surge2_in, surge2_in], dim=1)
        hidden2 = self.fc2(surge2)
        hidden = torch.cat([hidden1, hidden2], dim=0).unsqueeze(0)
        cell = cell + hidden
        c_0 = torch.cat([cell]*self.lstm_layers, dim=0)
        h_0 = torch.randn(self.lstm_layers, c_0.shape[1], 1)
        
        lstm_input = torch.cat([t_surge1_out, t_surge2_out], dim=0).unsqueeze(2)
        output, _ = self.lstm(lstm_input, (h_0, c_0))
        output = output.squeeze()
        return output[:len(slp)], output[len(slp):]

In [13]:
loss_weights = torch.linspace(1, 0.1, 10, requires_grad=False).to(device)
    
def surge_prediction_metric(surge1_true, surge2_true, surge1_pred, surge2_pred):
    surge1_score = torch.mean(torch.square(surge1_true - surge1_pred) * loss_weights)
    surge2_score = torch.mean(torch.square(surge2_true - surge2_pred) * loss_weights)
    return surge1_score + surge2_score

In [14]:
def train(model, loss_fn, optmiser, epochs):
    train_loss = []
    train_acc = []
    
    for epoch_num in range(epochs):
        model.train()
        running_loss = []

        for batch, (slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, surge1_out, t_surge2_out, surge2_out) in enumerate(train_dataloader):
            slp = slp.to(device)
            t_surge1_in = t_surge1_in.to(device)
            surge1_in = surge1_in.to(device)
            t_surge2_in = t_surge2_in.to(device)
            surge2_in = surge2_in.to(device)
            t_surge1_out = t_surge1_out.to(device)
            surge1_out = surge1_out.to(device)
            t_surge2_out = t_surge2_out.to(device)
            surge2_out = surge2_out.to(device)
            surge1_out_pred, surge2_out_pred = model(slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out)
            
            loss = loss_fn(surge1_out, surge2_out, surge1_out_pred, surge2_out_pred)
            running_loss.append(loss.item())
            
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            
        epoch_loss = np.mean(running_loss)
        train_loss.append(epoch_loss)
        print(f'Epoch {epoch_num+1:03d} | Loss: {epoch_loss:.4f}')
        
        if epoch_num % 5 == 0:
            val_loss = evaluate(model, loss_fn)
            print(f'\tVal loss: {val_loss:.4f}', end=' ')
            if val_loss < .52:
                torch.save(model.state_dict(), str(model) + f'_{epoch_num}.pth')
                print('saved!')
            else:
                print()
            
    return train_loss

def evaluate(model, loss_fn):
    with torch.no_grad():
        model.eval()
        losses = []
        for slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, surge1_out, t_surge2_out, surge2_out in val_dataloader:
            slp = slp.to(device)
            t_surge1_in = t_surge1_in.to(device)
            surge1_in = surge1_in.to(device)
            t_surge2_in = t_surge2_in.to(device)
            surge2_in = surge2_in.to(device)
            t_surge1_out = t_surge1_out.to(device)
            surge1_out = surge1_out.to(device)
            t_surge2_out = t_surge2_out.to(device)
            surge2_out = surge2_out.to(device)
            surge1_out_pred, surge2_out_pred = model(slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out)
            
            loss = loss_fn(surge1_out, surge2_out, surge1_out_pred, surge2_out_pred)
            losses.append(loss.item())
        return np.mean(losses)

In [312]:
resnet18 = Network(18, 100, 1).to(device)
optimiser = optim.Adam(resnet18.parameters())
_ = train(resnet18, surge_prediction_metric, optimiser, epochs=100)

Epoch 001 | Loss: 0.9974
	Val loss: 0.8707 
Epoch 002 | Loss: 0.8157
Epoch 003 | Loss: 0.7021
Epoch 004 | Loss: 0.6705
Epoch 005 | Loss: 0.6458
Epoch 006 | Loss: 0.6238
	Val loss: 0.6041 
Epoch 007 | Loss: 0.6120
Epoch 008 | Loss: 0.5934
Epoch 009 | Loss: 0.5767
Epoch 010 | Loss: 0.5643
Epoch 011 | Loss: 0.5542
	Val loss: 0.5512 
Epoch 012 | Loss: 0.5388
Epoch 013 | Loss: 0.5298
Epoch 014 | Loss: 0.5188
Epoch 015 | Loss: 0.5061
Epoch 016 | Loss: 0.5006
	Val loss: 0.5787 
Epoch 017 | Loss: 0.4950
Epoch 018 | Loss: 0.4706
Epoch 019 | Loss: 0.4635
Epoch 020 | Loss: 0.4596
Epoch 021 | Loss: 0.4483
	Val loss: 0.5558 
Epoch 022 | Loss: 0.4345
Epoch 023 | Loss: 0.4370
Epoch 024 | Loss: 0.4279
Epoch 025 | Loss: 0.4355
Epoch 026 | Loss: 0.4284
	Val loss: 0.5327 
Epoch 027 | Loss: 0.4245
Epoch 028 | Loss: 0.4101
Epoch 029 | Loss: 0.3909
Epoch 030 | Loss: 0.4134
Epoch 031 | Loss: 0.3911
	Val loss: 0.5426 
Epoch 032 | Loss: 0.3980
Epoch 033 | Loss: 0.4332
Epoch 034 | Loss: 0.4151
Epoch 035 | Loss:

In [276]:
resnet34 = Network(34, 100).to(device)
optimiser = optim.Adam(resnet34.parameters())
_ = train(resnet34, surge_prediction_metric, optimiser, epochs=100)
torch.save(resnet34.state_dict(), 'resnet34_100.pth')

Epoch 001 | Loss: 1.0114
	Val loss: 0.8565
Epoch 002 | Loss: 0.7872
Epoch 003 | Loss: 0.6887
Epoch 004 | Loss: 0.6610
Epoch 005 | Loss: 0.6327
Epoch 006 | Loss: 0.6371
	Val loss: 0.6249
Epoch 007 | Loss: 0.6212
Epoch 008 | Loss: 0.6068
Epoch 009 | Loss: 0.6165
Epoch 010 | Loss: 0.6224
Epoch 011 | Loss: 0.6105
	Val loss: 0.5638
Epoch 012 | Loss: 0.5944
Epoch 013 | Loss: 0.5824
Epoch 014 | Loss: 0.5961
Epoch 015 | Loss: 0.6069
Epoch 016 | Loss: 0.6043
	Val loss: 0.5853
Epoch 017 | Loss: 0.6250
Epoch 018 | Loss: 0.6136
Epoch 019 | Loss: 0.5964
Epoch 020 | Loss: 0.5831
Epoch 021 | Loss: 0.5742
	Val loss: 0.5372
Epoch 022 | Loss: 0.5690
Epoch 023 | Loss: 0.5690
Epoch 024 | Loss: 0.5611
Epoch 025 | Loss: 0.5555
Epoch 026 | Loss: 0.5572
	Val loss: 0.5285
Epoch 027 | Loss: 0.5504
Epoch 028 | Loss: 0.5456
Epoch 029 | Loss: 0.5428
Epoch 030 | Loss: 0.5438
Epoch 031 | Loss: 0.5359
	Val loss: 0.5194
Epoch 032 | Loss: 0.5350
Epoch 033 | Loss: 0.5391
Epoch 034 | Loss: 0.5378
Epoch 035 | Loss: 0.5353

In [328]:
resnet34 = Network(34, 100, 1).to(device)
optimiser = optim.Adam(resnet34.parameters())
_ = train(resnet34, surge_prediction_metric, optimiser, epochs=100)

Epoch 001 | Loss: 1.0465
	Val loss: 0.9183 


KeyboardInterrupt: 

In [324]:
resnet34 = Network(34, 100, 1).to(device)
optimiser = optim.Adam(resnet34.parameters())
_ = train(resnet34, surge_prediction_metric, optimiser, epochs=100)

Epoch 001 | Loss: 1.0118
	Val loss: 0.8851 
Epoch 002 | Loss: 0.9092
Epoch 003 | Loss: 0.8304
Epoch 004 | Loss: 0.7378
Epoch 005 | Loss: 0.6844
Epoch 006 | Loss: 0.6760
	Val loss: 0.7541 
Epoch 007 | Loss: 0.7078
Epoch 008 | Loss: 0.6507
Epoch 009 | Loss: 0.6355
Epoch 010 | Loss: 0.6295
Epoch 011 | Loss: 0.6365
	Val loss: 0.6262 
Epoch 012 | Loss: 0.6342
Epoch 013 | Loss: 0.6089
Epoch 014 | Loss: 0.6008
Epoch 015 | Loss: 0.6169
Epoch 016 | Loss: 0.6034
	Val loss: 0.5728 
Epoch 017 | Loss: 0.5911
Epoch 018 | Loss: 0.5845
Epoch 019 | Loss: 0.5910
Epoch 020 | Loss: 0.5797
Epoch 021 | Loss: 0.5702
	Val loss: 0.5586 
Epoch 022 | Loss: 0.5746
Epoch 023 | Loss: 0.5759
Epoch 024 | Loss: 0.5822
Epoch 025 | Loss: 0.5663
Epoch 026 | Loss: 0.5722
	Val loss: 0.5673 
Epoch 027 | Loss: 0.5856
Epoch 028 | Loss: 0.5588
Epoch 029 | Loss: 0.5605
Epoch 030 | Loss: 0.5775
Epoch 031 | Loss: 0.5729
	Val loss: 0.5425 
Epoch 032 | Loss: 0.5576
Epoch 033 | Loss: 0.5556
Epoch 034 | Loss: 0.5493
Epoch 035 | Loss:

In [316]:
resnet34 = Network(34, 100, 2).to(device)
optimiser = optim.Adam(resnet34.parameters())
_ = train(resnet34, surge_prediction_metric, optimiser, epochs=100)

Epoch 001 | Loss: 0.9736
	Val loss: 0.7503 
Epoch 002 | Loss: 0.7425
Epoch 003 | Loss: 0.6894
Epoch 004 | Loss: 0.6611
Epoch 005 | Loss: 0.6381
Epoch 006 | Loss: 0.6233
	Val loss: 0.5656 
Epoch 007 | Loss: 0.6172
Epoch 008 | Loss: 0.5983
Epoch 009 | Loss: 0.5917
Epoch 010 | Loss: 0.5795
Epoch 011 | Loss: 0.5689
	Val loss: 0.5315 
Epoch 012 | Loss: 0.5609
Epoch 013 | Loss: 0.5613
Epoch 014 | Loss: 0.5471
Epoch 015 | Loss: 0.5461
Epoch 016 | Loss: 0.5674
	Val loss: 0.5252 
Epoch 017 | Loss: 0.5407
Epoch 018 | Loss: 0.5311
Epoch 019 | Loss: 0.5210
Epoch 020 | Loss: 0.5128
Epoch 021 | Loss: 0.5005
	Val loss: 0.5267 
Epoch 022 | Loss: 0.4945
Epoch 023 | Loss: 0.4838
Epoch 024 | Loss: 0.4846
Epoch 025 | Loss: 0.4796
Epoch 026 | Loss: 0.4699
	Val loss: 0.5222 
Epoch 027 | Loss: 0.4653
Epoch 028 | Loss: 0.4508
Epoch 029 | Loss: 0.4373
Epoch 030 | Loss: 0.4390
Epoch 031 | Loss: 0.4390
	Val loss: 0.5146 saved!
Epoch 032 | Loss: 0.4343
Epoch 033 | Loss: 0.4401
Epoch 034 | Loss: 0.4990
Epoch 035 |

In [317]:
resnet34 = Network(34, 100, 3).to(device)
optimiser = optim.Adam(resnet34.parameters())
_ = train(resnet34, surge_prediction_metric, optimiser, epochs=100)

Epoch 001 | Loss: 1.0223
	Val loss: 0.8374 
Epoch 002 | Loss: 0.7569
Epoch 003 | Loss: 0.6864
Epoch 004 | Loss: 0.6630
Epoch 005 | Loss: 0.6383
Epoch 006 | Loss: 0.6348
	Val loss: 0.6230 
Epoch 007 | Loss: 0.6353
Epoch 008 | Loss: 0.6172
Epoch 009 | Loss: 0.6092
Epoch 010 | Loss: 0.6021
Epoch 011 | Loss: 0.5911
	Val loss: 0.5639 
Epoch 012 | Loss: 0.5861
Epoch 013 | Loss: 0.5794
Epoch 014 | Loss: 0.5680
Epoch 015 | Loss: 0.5596
Epoch 016 | Loss: 0.5542
	Val loss: 0.5353 
Epoch 017 | Loss: 0.5504
Epoch 018 | Loss: 0.5418
Epoch 019 | Loss: 0.5330
Epoch 020 | Loss: 0.5321
Epoch 021 | Loss: 0.5249
	Val loss: 0.5283 
Epoch 022 | Loss: 0.5157
Epoch 023 | Loss: 0.5065
Epoch 024 | Loss: 0.4972
Epoch 025 | Loss: 0.5113
Epoch 026 | Loss: 0.5004
	Val loss: 0.5200 
Epoch 027 | Loss: 0.4843
Epoch 028 | Loss: 0.4913
Epoch 029 | Loss: 0.5162
Epoch 030 | Loss: 0.4952
Epoch 031 | Loss: 0.4721
	Val loss: 0.5544 
Epoch 032 | Loss: 0.4640
Epoch 033 | Loss: 0.4625
Epoch 034 | Loss: 0.4629
Epoch 035 | Loss:

In [318]:
resnet34 = Network(34, 50, 1).to(device)
optimiser = optim.Adam(resnet34.parameters())
_ = train(resnet34, surge_prediction_metric, optimiser, epochs=100)

Epoch 001 | Loss: 1.0296
	Val loss: 0.9139 
Epoch 002 | Loss: 0.9456
Epoch 003 | Loss: 0.8634
Epoch 004 | Loss: 0.7983
Epoch 005 | Loss: 0.7284
Epoch 006 | Loss: 0.7102
	Val loss: 0.8165 
Epoch 007 | Loss: 0.7362
Epoch 008 | Loss: 0.6817
Epoch 009 | Loss: 0.6596
Epoch 010 | Loss: 0.6511
Epoch 011 | Loss: 0.6476
	Val loss: 0.5899 
Epoch 012 | Loss: 0.6237
Epoch 013 | Loss: 0.6120
Epoch 014 | Loss: 0.6107
Epoch 015 | Loss: 0.6139
Epoch 016 | Loss: 0.5983
	Val loss: 0.5733 
Epoch 017 | Loss: 0.5945
Epoch 018 | Loss: 0.5961
Epoch 019 | Loss: 0.5841
Epoch 020 | Loss: 0.5795
Epoch 021 | Loss: 0.5814
	Val loss: 0.5545 
Epoch 022 | Loss: 0.5734
Epoch 023 | Loss: 0.5656
Epoch 024 | Loss: 0.5635
Epoch 025 | Loss: 0.6119
Epoch 026 | Loss: 0.5896
	Val loss: 0.5537 
Epoch 027 | Loss: 0.5729
Epoch 028 | Loss: 0.5640
Epoch 029 | Loss: 0.5544
Epoch 030 | Loss: 0.5503
Epoch 031 | Loss: 0.5496
	Val loss: 0.5503 
Epoch 032 | Loss: 0.5456
Epoch 033 | Loss: 0.5358
Epoch 034 | Loss: 0.5536
Epoch 035 | Loss:

In [319]:
resnet34 = Network(34, 200, 1).to(device)
optimiser = optim.Adam(resnet34.parameters())
_ = train(resnet34, surge_prediction_metric, optimiser, epochs=100)

Epoch 001 | Loss: 1.0229
	Val loss: 0.8867 
Epoch 002 | Loss: 0.9117
Epoch 003 | Loss: 0.8263
Epoch 004 | Loss: 0.7479
Epoch 005 | Loss: 0.6765
Epoch 006 | Loss: 0.6641
	Val loss: 0.5946 
Epoch 007 | Loss: 0.6396
Epoch 008 | Loss: 0.6184
Epoch 009 | Loss: 0.6027
Epoch 010 | Loss: 0.5961
Epoch 011 | Loss: 0.5866
	Val loss: 0.5634 
Epoch 012 | Loss: 0.5791
Epoch 013 | Loss: 0.5643
Epoch 014 | Loss: 0.5540
Epoch 015 | Loss: 0.5434
Epoch 016 | Loss: 0.5384
	Val loss: 0.5388 
Epoch 017 | Loss: 0.5298
Epoch 018 | Loss: 0.5143
Epoch 019 | Loss: 0.5047
Epoch 020 | Loss: 0.4959
Epoch 021 | Loss: 0.4907
	Val loss: 0.5695 
Epoch 022 | Loss: 0.4836
Epoch 023 | Loss: 0.4832
Epoch 024 | Loss: 0.4649
Epoch 025 | Loss: 0.4545
Epoch 026 | Loss: 0.4457
	Val loss: 0.5440 
Epoch 027 | Loss: 0.4385
Epoch 028 | Loss: 0.4377
Epoch 029 | Loss: 0.4466
Epoch 030 | Loss: 0.4211
Epoch 031 | Loss: 0.4123
	Val loss: 0.5603 
Epoch 032 | Loss: 0.4073
Epoch 033 | Loss: 0.4031
Epoch 034 | Loss: 0.3929
Epoch 035 | Loss:

In [320]:
resnet50 = Network(50, 100, 1).to(device)
optimiser = optim.Adam(resnet50.parameters())
_ = train(resnet50, surge_prediction_metric, optimiser, epochs=100)

Epoch 001 | Loss: 1.0308
	Val loss: 0.8777 
Epoch 002 | Loss: 0.8113
Epoch 003 | Loss: 0.7012
Epoch 004 | Loss: 0.6576
Epoch 005 | Loss: 0.6321
Epoch 006 | Loss: 0.6288
	Val loss: 0.5834 
Epoch 007 | Loss: 0.6115
Epoch 008 | Loss: 0.6023
Epoch 009 | Loss: 0.6059
Epoch 010 | Loss: 0.6204
Epoch 011 | Loss: 0.6071
	Val loss: 0.5749 
Epoch 012 | Loss: 0.5898
Epoch 013 | Loss: 0.5818
Epoch 014 | Loss: 0.5839
Epoch 015 | Loss: 0.5724
Epoch 016 | Loss: 0.5579
	Val loss: 0.5337 
Epoch 017 | Loss: 0.5519
Epoch 018 | Loss: 0.5438
Epoch 019 | Loss: 0.5401
Epoch 020 | Loss: 0.5246
Epoch 021 | Loss: 0.5132
	Val loss: 0.6028 
Epoch 022 | Loss: 0.5152
Epoch 023 | Loss: 0.5400
Epoch 024 | Loss: 0.5886
Epoch 025 | Loss: 0.5581
Epoch 026 | Loss: 0.5496
	Val loss: 0.5551 
Epoch 027 | Loss: 0.5437
Epoch 028 | Loss: 0.5293
Epoch 029 | Loss: 0.5289
Epoch 030 | Loss: 0.6016
Epoch 031 | Loss: 0.5634
	Val loss: 0.5577 
Epoch 032 | Loss: 0.5501
Epoch 033 | Loss: 0.5268
Epoch 034 | Loss: 0.5263
Epoch 035 | Loss:

In [331]:
def write_submission(model, fn='submission.csv'):
    COLUMNS = [
        'surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4',
        'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9',
        'surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4',
        'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9' ]
    surge1_output = []
    surge2_output = []

    with torch.no_grad():
        for slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out in test_dataloader:
            slp = slp.to(device)
            t_surge1_in = t_surge1_in.to(device)
            surge1_in = surge1_in.to(device)
            t_surge2_in = t_surge2_in.to(device)
            surge2_in = surge2_in.to(device)
            t_surge1_out = t_surge1_out.to(device)
            t_surge2_out = t_surge2_out.to(device)
            surge1_out_pred, surge2_out_pred = model(slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out)
            surge1_output.append(surge1_out_pred)
            surge2_output.append(surge2_out_pred)

    surge1_output = torch.cat(surge1_output, dim=0)
    surge2_output = torch.cat(surge2_output, dim=0)
    test_outputs = torch.cat([surge1_output, surge2_output], dim=1).detach().cpu().numpy()

    test_df = pd.DataFrame(test_outputs, index=test_id_seq, columns=COLUMNS)
    test_df.index.name = 'id_sequence'
    test_df.to_csv(fn)
    return test_df

In [336]:
model_fn = 'resnet34_100_2_30.pth'
model = Network(34, 100, 2)
model.load_state_dict(torch.load(model_fn, map_location=device))
model = model.to(device)
df = write_submission(model)

In [266]:
df

,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,surge1_t9,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
id_sequence,,,,,,,,,,,,,,,,,,,,
5600,-1.522672,-1.640056,-1.812372,-1.873276,-1.911106,-1.892778,-1.814209,-1.698588,-1.570127,-1.445986,-1.077388,-1.206931,-1.322765,-1.355515,-1.416860,-1.485878,-1.503089,-1.474643,-1.412876,-1.315216
5601,-0.172018,-0.244204,-0.333128,-0.247466,0.008339,0.311038,0.669055,1.009192,1.198229,1.189028,0.016760,-0.052519,-0.104194,-0.077168,0.124576,0.411364,0.781786,1.110761,1.277642,1.251851
5602,-0.728413,-1.103798,-1.081307,-0.869735,-0.684275,-0.531688,-0.410095,-0.311700,-0.223303,-0.145472,1.353438,1.628443,1.170808,0.442155,0.039545,-0.142167,-0.216019,-0.203616,-0.161684,-0.113963
5603,-0.214031,0.022230,0.161170,0.047790,-0.173768,-0.312626,-0.312222,-0.243562,-0.164647,-0.095614,1.032095,1.588513,1.518492,0.934075,0.517910,0.386632,0.364263,0.341955,0.313909,0.286394
5604,0.188806,0.256014,0.136325,-0.055788,-0.255821,-0.438868,-0.531611,-0.504565,-0.409072,-0.300028,-0.038583,0.077969,-0.076706,-0.310813,-0.561307,-0.757425,-0.837745,-0.798326,-0.687479,-0.554552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,-0.202868,-0.458387,-0.676621,-0.850127,-0.801504,-0.661781,-0.550004,-0.469796,-0.375337,-0.267382,0.679106,-0.081019,-0.426764,-0.630051,-0.645582,-0.500996,-0.302407,-0.138851,-0.036838,0.022590
6105,0.004178,-0.027268,-0.104875,-0.129565,-0.042405,0.124772,0.274499,0.309280,0.270955,0.226377,0.155758,-0.343349,-0.494503,-0.406895,-0.211891,0.009162,0.194007,0.265217,0.251194,0.218806
6106,-0.429599,-0.720932,-0.728523,-0.629140,-0.443044,-0.221718,0.056978,0.360464,0.566990,0.590129,0.005821,-0.308064,-0.262538,-0.246235,-0.120654,0.120976,0.452356,0.784378,0.954932,0.906470


In [337]:
df

,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,surge1_t9,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
id_sequence,,,,,,,,,,,,,,,,,,,,
5600,-1.199508,-1.171856,-1.131088,-1.066332,-0.990663,-0.864091,-0.801858,-0.746576,-0.696449,-0.651072,-0.726454,-0.739934,-0.704749,-0.652886,-0.602378,-0.553215,-0.507450,-0.465277,-0.426380,-0.390044
5601,-0.189759,-0.080710,0.080381,0.214584,0.307948,0.374662,0.426763,0.470630,0.509728,0.546062,-0.023364,0.019109,0.151435,0.216138,0.207174,0.181843,0.155302,0.228899,0.305990,0.377340
5602,-0.236023,-0.227396,-0.211098,-0.195290,-0.173217,-0.148450,-0.123184,-0.097737,-0.071892,-0.045371,0.345954,0.105727,0.012435,-0.008099,0.004825,0.026204,0.049977,0.044051,0.072038,0.103419
5603,-0.019892,-0.034730,-0.047265,-0.054631,-0.026954,0.002579,0.007071,-0.003261,0.008565,0.019975,0.561087,0.464187,0.381445,0.333398,0.350693,0.344865,0.275331,0.209318,0.188466,0.206171
5604,0.047979,0.138255,0.121141,0.073845,0.050825,0.023558,0.002464,-0.011403,-0.019011,-0.021168,-0.131821,0.035536,0.045335,0.005460,-0.030595,-0.056825,-0.075062,-0.067410,-0.075035,-0.065493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,-0.135019,-0.469348,-0.601718,-0.616191,-0.556177,-0.512051,-0.472327,-0.434713,-0.399322,-0.341437,1.137056,0.598249,0.287585,0.139036,0.090300,0.081051,0.086205,0.098205,0.114662,0.097834
6105,0.683089,0.908907,0.897307,0.772712,0.645870,0.534164,0.454687,0.378453,0.335563,0.306968,1.416681,1.488015,1.375669,1.207726,1.055663,0.932966,0.837150,0.763569,0.708463,0.595871
6106,-0.003328,-0.161861,-0.211071,-0.234781,-0.243938,-0.244648,-0.240823,-0.234090,-0.225040,-0.213867,0.487938,0.265363,0.157881,0.095710,0.056991,0.033530,0.024331,0.018640,0.017186,0.020715


## Benchmark
Train using kNN of pressure fields at two instants in time, with 40 neighbours

In [4]:
def surge_prediction_metric(dataframe_y_true, dataframe_y_pred):
    weights = np.linspace(1, 0.1, 10)[np.newaxis]
    surge1_columns = [
        'surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4',
        'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9' ]
    surge2_columns = [
        'surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4',
        'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9' ]
    surge1_score = (weights * (dataframe_y_true[surge1_columns].values - dataframe_y_pred[surge1_columns].values)**2).mean()
    surge2_score = (weights * (dataframe_y_true[surge2_columns].values - dataframe_y_pred[surge2_columns].values)**2).mean()

    return surge1_score + surge2_score

In [5]:
nfields = 2; time_step_slp = 8
slp_train = []
slp_all = X_train['slp']
for i in range(5559):
    slp_train.append(np.ndarray.flatten(slp_all[i,-1]))
    for j in range(1,nfields):
        slp_train[-1] = np.concatenate( ( slp_train[-1], np.ndarray.flatten(slp_all[i,-1-j*time_step_slp]) ) )
slp_train = np.array(slp_train)

In [6]:
slp_test = []
slp_all_test = X_test['slp']
for i in range(509):
    slp_test.append(np.ndarray.flatten(slp_all_test[i,-1]))
    for j in range(1,nfields):
        slp_test[-1] = np.concatenate( ( slp_test[-1], np.ndarray.flatten(slp_all_test[i,-1-j*time_step_slp]) ) )
slp_test = np.array(slp_test)

In [7]:
tree = BallTree(slp_train)

In [8]:
surge_test_benchmark = []; k = 40
for i in range(509):
    dist, ind = tree.query([slp_test[i]], k=k)
    surge_test_benchmark.append(np.mean(surge_train[ind[0]], axis=0))
surge_test_benchmark = np.array(surge_test_benchmark)

In [9]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]
y_test_benchmark = pd.DataFrame(data=surge_test_benchmark, columns=y_columns, index=X_test['id_sequence'])
y_test_benchmark.to_csv('Y_test_benchmark.csv', index_label='id_sequence', sep=',')